# A small convnet from scratch

From: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

We will use a very small convnet with few layers and few filters per layer, alongside data augmentation and dropout. Dropout also helps reduce overfitting, by preventing a layer from seeing twice the exact same pattern, thus acting in a way analoguous to data augmentation (you could say that both dropout and data augmentation tend to disrupt random correlations occuring in your data).

The code snippet below is our first model, a simple stack of 3 convolution layers with a ReLU activation and followed by max-pooling layers. This is very similar to the architectures that Yann LeCun advocated in the 1990s for image classification (with the exception of ReLU).

In [13]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [36]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

On top of it we stick two fully-connected layers. We end the model with a single unit and a sigmoid activation, which is perfect for a binary classification. To go with it we will also use the binary_crossentropy loss to train our model.

In [37]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [38]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 7222 images belonging to 2 classes.
Found 888 images belonging to 2 classes.


In [39]:
model.fit_generator(
        train_generator,
        samples_per_epoch=2000,
        nb_epoch=10,
        validation_data=validation_generator,
        nb_val_samples=800)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/10
1984/2000 [============================>.] - ETA: 0s - loss: 0.7110 - acc: 0.5131

/Users/wattx/anaconda/envs/py3/lib/python3.5/site-packages/keras/engine/training.py:1527: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


2016/2000 [==============================] - 124s - loss: 0.7108 - acc: 0.5119 - val_loss: 0.6919 - val_acc: 0.5288
Epoch 2/10
2016/2000 [==============================] - 103s - loss: 0.6878 - acc: 0.5635 - val_loss: 0.6841 - val_acc: 0.5558
Epoch 3/10
2016/2000 [==============================] - 102s - loss: 0.6757 - acc: 0.5784 - val_loss: 0.6554 - val_acc: 0.5995
Epoch 4/10
2006/2000 [==============================] - 100s - loss: 0.6653 - acc: 0.5872 - val_loss: 0.6356 - val_acc: 0.6092
Epoch 5/10
2016/2000 [==============================] - 104s - loss: 0.6598 - acc: 0.6017 - val_loss: 0.6324 - val_acc: 0.6226
Epoch 6/10
2016/2000 [==============================] - 100s - loss: 0.6442 - acc: 0.6270 - val_loss: 0.6079 - val_acc: 0.6590
Epoch 7/10
2016/2000 [==============================] - 102s - loss: 0.6186 - acc: 0.6553 - val_loss: 0.6016 - val_acc: 0.6481
Epoch 8/10
2006/2000 [==============================] - 100s - loss: 0.5843 - acc: 0.6830 - val_loss: 0.5989 - val_acc: 0.